In [79]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    DiscreteContactApproximation,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    RigidTransform,
    RotationMatrix,
    Solve,
    StartMeshcat,
    WeldJoint,
    eq,
    FrameIndex,
    JointIndex
)

from manipulation import ConfigureParser, running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad

In [69]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7005


## Problem Description
We will solve inverse kinematics for one arm on a PR2 robot. This robot model has been modified to have convex collision geometries, and to simplify this problem, we have welded (fixed) several joints that are irrelevent to the kinematics of the arms.

In [80]:
# TODO(russt): Move this to Drake.


def ReplaceJointWithWeld(plant, joint):
    for actuator in [
        plant.get_joint_actuator(index) for index in plant.GetJointActuatorIndices()
    ]:
        if actuator.joint() == joint:
            plant.RemoveJointActuator(actuator)

    weld = WeldJoint(
        joint.name(), joint.frame_on_parent(), joint.frame_on_child(), RigidTransform()
    )
    plant.RemoveJoint(joint)
    plant.AddJoint(weld)


def build_env():
    """Load in models and build the diagram."""
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)
    plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://manipulation/pr2_shelves.dmd.yaml")
    # Remove mimic constraints:
    for id in plant.GetConstraintIds():
        plant.RemoveConstraint(id)
    # Remove some superfluous joints
    ReplaceJointWithWeld(plant, plant.GetJointByName("head_pan_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("head_tilt_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_l_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_r_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_l_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_r_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_l_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_r_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_l_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_r_finger_tip_joint"))
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(
        builder,
        scene_graph.get_query_output_port(),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    diagram = builder.Build()
    return diagram, plant, scene_graph

In [81]:
goal_rotation = RotationMatrix(
    [
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1],
    ]
)
goal_position = np.array([-0.83, 0.18, 1.4])
goal_pose = RigidTransform(goal_rotation, goal_position)

### IK for a Mobile Manipulator

Given a RigidTransform X_WG, compute a robot configuration placing the left gripper at that pose. We use optimization to solve the IK problem, and we repeatedly solve the program with random initializations until it succeeds. We have implemented a skeleton of the necessary code in the following function, but you must complete several pieces:

- Add position and orientation constraints to the gripper frame. The end effector should match the desired pose to within 1mm translation along each axis, and the rotation should be off by no more than 1 degree.
- If `fix_base` is True, constrain the base pose $(x,y,\theta)$ to be equal to `base_pose`.
- Add a collision free constraint with [AddMinimumDistanceLowerBoundConstraint](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html#a2ecd71efd675a7e1a4293adb05c9b9df). The minimum distance between any pair of collision geometries should be at least 1cm.
- Compute a random initial guess for the joint angles within the robot's joint limits. You can access the joint limits from the multibody plant, but some of the joints are angle-valued and don't have limits. For these joints, use the range $[-\pi,\pi]$.

In [127]:
def solve_ik(X_WG, max_tries=10, fix_base=False, base_pose=np.zeros(3)):
    diagram, plant, scene_graph = build_env()

    plant.GetFrameByName("l_gripper_palm_link")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    scene_graph.GetMyContextFromRoot(context)

    # Note: passing in a plant_context is necessary for collision-free constraints!
    ik = InverseKinematics(plant, plant_context, with_joint_limits=True)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram
    q_nominal = np.zeros(len(q_variables))
    q_nominal[0:3] = base_pose
    prog.AddQuadraticErrorCost(np.eye(len(q_variables)), q_nominal, q_variables)

    # Add your constraints here

    # gripper tolerances
    gripper_tol_p = 0.001 # position constraint, m
    gripper_tol_R = 1 # rotation constraint, deg

    # gripper position constraints
    # need to place the end effector, not the palm link
    ik.AddPositionConstraint(
        plant.GetFrameByName("l_gripper_palm_link"),
        np.zeros(3),
        plant.world_frame(),
        X_WG.translation() - gripper_tol_p,
        X_WG.translation() + gripper_tol_p
    )
    ik.AddOrientationConstraint(
        plant.GetFrameByName("l_gripper_palm_link"),
        RotationMatrix(),
        plant.world_frame(),
        X_WG.rotation(),
        gripper_tol_R * np.pi / 180
    )

    # mobile manipulator base constraints
    if fix_base:
        ik.AddPositionConstraint(
            plant.GetFrameByName("base_link"),
            np.zeros(3),
            plant.world_frame(),
            base_pose - 0.001,
            base_pose + 0.001,
        )
        ik.AddOrientationConstraint(
            plant.GetFrameByName("base_link"),
            RotationMatrix(),
            plant.world_frame(),
            RotationMatrix(),
            0.001,
        )
    
    # collision free constraint
    ik.AddMinimumDistanceLowerBoundConstraint(0.01)

    for count in range(max_tries):
        # Compute a random initial guess here
        q_guess = np.random.randn(len(q_variables))
        prog.SetInitialGuess(q_variables, q_guess)

        result = Solve(prog)

        if running_as_notebook:
            render_context = diagram.CreateDefaultContext()
            plant.SetPositions(
                plant.GetMyContextFromRoot(render_context),
                result.GetSolution(q_variables),
            )
            diagram.ForcedPublish(context)

        if result.is_success():
            print("Succeeded in %d tries!" % (count + 1))
            return result.GetSolution(q_variables)

    print("Failed!")
    return None

In [128]:
# get all the frames from the diagram (helper debugging function)
def get_frames(plant):
    frames = []
    for i in range(plant.num_frames()):
        frame = plant.get_frame(FrameIndex(i))
        frames.append(frame.name())
    return frames

# get_frames(plant)

# get the joint limits for the PR2 from the plant
def get_joint_limits(plant):
    joint_limits = {}
    for i in range(plant.num_joints()):
        try:
            joint = plant.get_joint(JointIndex(i))
            joint_limits[joint.name()] = (joint.position_lower_limits(), joint.position_upper_limits())
        except:
            continue
    return joint_limits

# get_frames(plant)
# get_joint_limits(plant)

First, we show an example where we have fixed the base. It may take many tries for it to solve, or not solve at all! At each iteration, we visualize where the optimizer stopped, so you can see what the failures look like.

In [131]:
AddMeshcatTriad(meshcat,"goal",length=0.2,radius=0.005,opacity=0.4,X_PT=goal_pose)

solve_ik(
    goal_pose,
    max_tries=20,
    fix_base=True,
    base_pose=np.array([-1.23, 0.05, 0]),
)

Failed!


When we allow the base to move freely, the inverse kinematics can be solved much more easily!

In [132]:
solve_ik(goal_pose, fix_base=False)

Succeeded in 1 tries!


array([-1.14495100e+00, -5.08897309e-01,  5.62910743e-01,  2.82462632e-01,
       -1.29731799e-01, -7.67307680e-02, -2.42311425e-03,  0.00000000e+00,
        2.54904694e-04,  0.00000000e+00, -1.18124439e-07,  4.81810882e-01,
       -3.94567068e-01,  1.04761343e+00, -4.32589357e-01,  9.71513526e-01,
       -7.92099118e-01,  7.15978536e-01])

In [133]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.mobile.test_mobile_base_ik import TestMobileBaseIk

Grader.grade_output([TestMobileBaseIk], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 5/5.

Score for Test solve_ik is 5/5.
- Succeeded in 2 tries!
Succeeded in 3 tries!
Succeeded in 2 tries!
Failed!



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=157857d4-6367-4a01-a572-0646c5286123' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>